<a href="https://colab.research.google.com/github/EgorPlay2020/VolFx/blob/master/notebooks/Tutorial_2_train_your_first_TTS_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучите свою первую 🐸 модель TTS 💫

### 👋 Здравствуйте и добро пожаловать в Coqui (🐸) TTS

Цель этой тетради - показать вам ** типичный рабочий процесс ** для ** обучения** и ** тестирования** модели TTS с помощью 🐸.

Давайте обучим очень маленькую модель на очень небольшом объеме данных, чтобы мы могли быстро выполнять итерации.

В этом блокноте мы:

1. Загрузим данные и отформатируем их для 🐸 TTS.
2. Настроим учебные и тестовые прогоны.
3. Обучим новую модель.
4. Протестируйте модель и продемонстрируйте ее работоспособность.

Итак, давайте сразу перейдем к делу!

In [9]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

## ✅ Подготовка данных

### **Перво-наперво **: нам нужны некоторые данные.

Мы тренируем модель преобразования текста в речь, поэтому нам нужен некоторый текст и немного речи. В частности, мы хотим, чтобы речь была переведена. Речь должна быть разделена на аудиоклипы, и каждый клип нуждается в транскрипции. Более подробную информацию о требованиях к данным, таких как характеристики записи, фоновый шум и словарный запас, можно найти в документации [🐸TTS](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

Если у вас есть один аудиофайл и вам необходимо разбить его на клипы. Также важно использовать формат аудиофайла без потерь, чтобы избежать артефактов сжатия. Мы рекомендуем использовать формат файла wav.

Формат данных, который мы будем использовать для этого урока, взят из широко используемого набора данных **LJSpeech**, где ** волны** собраны в папке:

<span style="color:purple;font-size:15px">
/wavs<br />
 &emsp;| - audio1.wav<br />
 &emsp;| - audio2.wav<br />
 &emsp;| - audio3.wav<br />
  ...<br />
</span>

и файл **metadata.csv** будет содержать название аудиофайла параллельно расшифровке, разделенное символом "|":

<span style="color:purple;font-size:15px">
# metadata.csv <br />
audio1|This is my sentence. <br />
audio2|This is maybe my sentence. <br />
audio3|This is certainly my sentence. <br />
audio4|Let this be your sentence. <br />
...
</span>

В итоге у нас должна получиться следующая структура папок**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br />
&emsp;| <br />
&emsp;| -> metadata.csv<br />
&emsp;| -> /wavs<br />
&emsp;&emsp;| -> audio1.wav<br />
&emsp;&emsp;| -> audio2.wav<br />
&emsp;&emsp;| ...<br />
</span>

🐸TTS уже предоставляет инструменты для работы с LJSPEECH. если вы используете тот же формат, вы можете сразу начать обучение своих моделей. <br />

После того, как вы соберете и отформатируете свой набор данных, вам нужно проверить две вещи. Нужны ли вам **_formatter_** и **_text_cleaner_**. <br /> **_formatter_** загружает текстовый файл (созданный выше) в виде списка, а **_text_cleaner_** выполняет последовательность операций нормализации текста, которая преобразует исходный текст в устное представление (например, преобразует числа в текст, сокращения и символы в устный формат)..

Если вы используете другой формат набора данных, чем LJSpeech или другие общедоступные наборы данных, которые поддерживает 🐸TTS, вам нужно написать свой собственный **_formatter_** и **_text_cleaner_**.

## ⏳️ Загрузка вашего набора данных
Загрузите один из наборов данных, поддерживаемых 🐸TTS.

Мы начнем с определения конфигурации набора данных и установки LJSpeech в качестве нашего целевого набора данных и определения пути к нему.

In [10]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)


In [3]:
# Download and extract LJSpeech dataset.

!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

--2025-01-31 16:35:27--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 138.199.46.65, 2400:52e0:1500::783:1
Connecting to data.keithito.com (data.keithito.com)|138.199.46.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [text/plain]
Saving to: ‘tts_train_dir/LJSpeech-1.1.tar.bz2’

tts_train_dir/LJSpe 100%[===================>]   2.56G  90.2MB/s    in 37s     

2025-01-31 16:36:05 (71.2 MB/s) - ‘tts_train_dir/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [2]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
)

NameError: name 'BaseDatasetConfig' is not defined

## ✅ Обучите новую модель

Давайте начнем тренировочный цикл 🚀🚀🚀.

Выбор архитектуры модели, которую вы хотели бы использовать, зависит от ваших потребностей и имеющихся ресурсов. Каждая архитектура модели имеет свои плюсы и минусы, которые определяют эффективность во время выполнения и качество передачи голоса.
У нас есть много рецептов в разделе "TTS/рецепты/", которые являются хорошей отправной точкой. В этом уроке мы будем использовать `GlowTTS`.

Мы начнем с инициализации конфигурации обучения модели.

In [15]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="basic_cleaners",
    use_phonemes=False,
    phoneme_language="ru",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Далее мы инициализируем аудиопроцессор, который используется для извлечения функций и ввода-вывода звука.

In [16]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


Далее мы инициализируем токенизатор, который используется для преобразования текста в последовательности идентификаторов токенов.  Если символы не определены в конфигурации, в конфигурацию передаются символы по умолчанию.

In [17]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Далее мы загрузим образцы данных. Каждый образец представляет собой список ``[текст, аудио_файл_пат, имя_языка]``. Вы можете настроить свой пользовательский загрузчик образцов, возвращающий список образцов.

In [18]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 13100 files in /content/tts_train_dir/LJSpeech-1.1


Теперь мы готовы к инициализации модели.

Модели используют объект config и менеджер динамиков в качестве входных данных. Конфигурация определяет такие детали модели, как количество слоев, размер встраиваемого элемента и т.д. Менеджер динамиков используется в моделях с несколькими динамиками.

In [19]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

Trainer предоставляет универсальный API для обучения всех моделей TTS со всеми его преимуществами, такими как смешанное обучение точности, распределенное обучение и т.д.

In [20]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir/run-January-31-2025_04+48PM-0000000

 > Model has 28597969 parameters


### И... 3,2,1... НАЧИНАЕМ ТРЕНИРОВАТЬСЯ 🚀🚀🚀

In [ ]:
trainer.fit()


 > EPOCH: 0/100
 --> tts_train_dir/run-January-31-2025_04+48PM-0000000
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

 > TRAINING (2025-01-31 16:48:43) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 12969
 | > Preprocessing samples
 | > Max text length: 188
 | > Min text length: 13
 | > Avg text length: 100.90014650319993
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24499.0
 | > Avg audio length: 144984.29755570978
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
truly replied, "yes.""where is it?" asked the governor.

 [!] Character '"' not found in the vocabulary. Discarding it. [!] Character '"' not found in the vocabulary. Discarding it.




   --> TIME: 2025-01-31 16:49:41 -- STEP: 0/406 -- GLOBAL_STEP: 0
     | > current_lr: 2.5e-07 
     | > step_time: 52.591  (52.59096121788025)
     | > loader_time: 5.488  (5.487969636917114)

 [!] `train_step()` retuned `None` outputs. Skipping training step.
 > Keyboard interrupt detected.
 > Saving model before exiting...

 > CHECKPOINT : tts_train_dir/run-January-31-2025_04+48PM-0000000/checkpoint_1.pth
 ! Run is kept in tts_train_dir/run-January-31-2025_04+48PM-0000000


#### 🚀 Запустите тензорную доску. 🚀
В записной книжке и на тензорной доске вы можете отслеживать ход выполнения вашей модели. Кроме того, на тензорной доске представлены определенные цифры и примеры выходных данных.

In [ ]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

## ✅ Протестируйте модель

Мы сделали это! 🙌

Давайте начнем тестирование, в ходе которого будут отображены показатели производительности.

Мы совершаем главный грех ML 😈 (он же - тестирование на наших обучающих данных), поэтому вы не хотите внедрять эту модель в производство. В этой записной книжке мы сосредоточились на самом рабочем процессе, так что это простительно.

Из результатов тестирования вы можете видеть, что наша крошечная модель адаптировалась к данным и в основном запомнила это предложение.

Когда вы начнете обучать свои собственные модели, убедитесь, что данные тестирования не включают в себя данные обучения 😅

Let's get the latest saved checkpoint.

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
 !tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

## 📣 Listen to the synthesized wave 📣

In [ ]:
import IPython
IPython.display.Audio("out.wav")

## 🎉 Поздравляем! 🎉 Теперь вы обучили свою первую модель TTS!
Следите за следующими уроками, чтобы изучить более продвинутый материал.